# Selection Tool for Hydrographic Equipment

Author: Freya Wittkugel, 05.11.2021

This tool will select suitable hydrographic sensors from an excel-file. 
The tool is valid for MBES, SBES, SBP and SSS. 
The information is up to date as stated in the xls-file. 

Run the tool by continuously pressing the play button. The blue line shows which cell is active. Please reset the variables after each run. Restart at section "Reset Variables". 

Run until section "User Input" and provide information as asked. 


## Reset Variables

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Import

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
from math import sqrt, pow
from ipywidgets import interact, interactive, fixed, interact_manual, FileUpload
import ipywidgets as widgets
from IPython.display import display
import io

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Functions for Selection by Survey Scenario

### MBES: mbes_scenario(depth), return df


In [3]:
def mbes_scenario(depth):
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'MBES':
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # draught is considered for minimum survey depth = transducer draught + 10% Spielraum. 
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]

    return df

### SBES: sbes_scenario(depth), return df

In [4]:
def sbes_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBES': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # draught is considered for minimum survey depth = transducer draught + 10% Spielraum. 
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]
    
    return df

### SBP: sbp_scenario(depth), return df

In [5]:
def sbp_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBP': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # Um klar vom nearfield zu sein, werden Tiefgang und mounting berücksichtigt = transducer draught + 10% Spielraum.
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]

    return df

### SSS: sss_scenario(depth), return df

In [6]:
def sss_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SSS': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    tow_height = 0.15*df.swath_width
    # depth_rating
    bedingung3 = (depth[1]-tow_height) < df.depth_rating
    
    # Maske mit Bedingungen auf df anwenden
    df = df[bedingung3]
    #print('bedingung1 erfüllt: ',bedingung1)
    #print('bedingung2 erfüllt: ', bedingung2)
    
    return df

## Step 2: Functions for Selection by Space Restrictions

### MBES: mbes_space(space_limits), return df

In [7]:
# df from first step, space limits from user input
def mbes_space(space_limits, df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # remember: sensor without dimension values are set to 0. They will be selected!
    # the transmitting array is arranged along the keel, so its length plus the rx perpendicular width is the limit
    bedingung1 = (df.tx_length+df.rx_width) <= length 
    # the receiving array is arranged perpendicular to the transmitting array (mills cross config) so its width is the limit
    bedingung2 = (df.rx_length) <= width
    # the weight for the whole transducer setup (tx+rx) shall not exceed the specification
    #bedingung3 = (df.tx_weight+df.rx_weight) <= weight 

    # apply to df
    #df = df[bedingung1 & bedingung2 & bedingung3]
    df = df[bedingung1 & bedingung2]
    #print('bedingung1 erfüllt: ',bedingung1)
    #print('bedingung2 erfüllt: ', bedingung2)
    #print('bedingung3 erfüllt: ', bedingung3)
    return df

### SBES: sbes_space(space_limits), return df

In [8]:
def sbes_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBES':
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    
    # apply to df
    df = df[bedingung1 & bedingung2 & bedingung3]
    
    return df

### SBP: sbp_space(space_limits), return df

In [9]:
def sbp_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBP':
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    
    # apply to df
    df = df[bedingung1 & bedingung2 & bedingung3]
    
    return df

### SSS: sss_space(space_limits), return df

In [10]:
def sss_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SSS': 
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    # nur fest installierte Geräte
    bedingung4 = (df.pole == 'no') & (df.towfish == 'no')
    df = df[bedingung1 & bedingung2 & bedingung3 & bedingung4]
    
    return df

## Widgets

In [11]:
# upload button
upload = FileUpload(accept= '.xlsx', multiple = False)

# widget for input of survey scenarios
scenario_select = widgets.SelectMultiple(
    options=['Bathymetric Survey', 'Seafloor Classification', 'Oceanographic Survey', 'Geophysical Survey'],
    value=[],
    rows=5,
    description='Purpose',
    disabled=False
)

# widget for input of desired survey depth
depth_range_slider = widgets.IntRangeSlider(
    value=[100,2000],
    min=0.2,
    max=11000,
    step=10,
    description='Survey Depths:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout={'width': '990px'}
)



## User Input

Enter your information here. 

- Upload the excel-file with information about sensors by clicking the upload button.

- Select the scenario(s) you want recommendations for. Multiple scenarios can be selected by holding strg. 

- Select the depth range of the survey area. Then click next cell. 

- Enter draught and dimensions available on the keel plate. 

Proceed with runnung the cells. 

In [14]:
box = widgets.VBox([upload, scenario_select, depth_range_slider])
display(box)

In [15]:
# ship_draught refers to depth of transducer surface
ship_draught = 3.71
# for the second step: maximum space available in mm, kg
keel_width = 1200
keel_length = 9000
weight_limit = 10000
space_limits = [keel_width, keel_length, weight_limit]

#### end of input

In [16]:
# read excel file
with open('Sensors_DataExcel.xlsx', 'wb') as output_file:
    for uploaded_filename in upload.value:
        content = upload.value[uploaded_filename]['content']
        output_file.write(content)

filename = 'Sensors.xlsx'
head = 4
xls = pd.ExcelFile(content)

# getting chosen scenario values
selected = scenario_select.value  
survey_scenario = []

for i in selected:
    survey_scenario.append(i)
    
# get values from slider
survey_depth = [depth_range_slider.value[0],depth_range_slider.value[1]]

27115

# Select Sensors by Survey Scenario

To receive recommended sensor models for the selected survey scenario please run the following sections. 

In [17]:
depth = [survey_depth]

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        #print('Instruments for nautical survey: ')
        for i in depth:
            nautical_mbes = mbes_scenario(i)
            nautical_sbes = sbes_scenario(i)
            nautical_sbp = sbp_scenario(i)
            
    if 'Seafloor Classification' in i:
        #print('Instruments for seabed investigation: ')
        for i in depth:
            seabed_mbes = mbes_scenario(i)
            seabed_sbp = sbp_scenario(i)
            seabed_sss = sss_scenario(i)
            
    if 'Oceanographic Survey' in i:
        #print('Instruments for investigation of the water column: ')
        for i in depth:
            oceanic_mbes = mbes_scenario(i) # hier noch Auswahlmöglichkeit einbauen, dass nur scientific ausgewählt werden
            #print('other not yet defined')
            
    if 'Geophysical Survey' in i:
        #print('Instruments for underground features: ')
        for i in depth:
            print('')
            

In [18]:
# prepare lists for next step

mbes_sensors = []
sbes_sensors = []
sbp_sensors = []
sss_sensors = []

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        mbes_sensors.append(nautical_mbes)
        sbes_sensors.append(nautical_sbes)
        sbp_sensors.append(nautical_sbp)
        #sss_sensors.append(nautical_sss)
    if 'Seabed Classification' in i:
        mbes_sensors.append(seabed_mbes)
        #sbes_sensors.append(seabed_sbes)
        sbp_sensors.append(seabed_sbp)
        sss_sensors.append(seabed_sss)
    if 'Oceanographic Survey' in i:
        mbes_sensors.append(oceanic_mbes)
        #sbes_sensors.append(oceanic_sbes)
        #sbp_sensors.append(oceanic_sbp)
        #sss_sensors.append(oceanic_sss)
    if 'Geophysical Survey' in i:
        #mbes_sensors.append(geophysical_mbes)
        #sbes_sensors.append(geophysical_sbes)
        #sbp_sensors.append(geophysical_sbp)
        #sss_sensors.append(geophysical_sss)
        pass

In [19]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates, only for not empty lists

if not mbes_sensors:
    all_mbes = []
else:
    all_mbes = pd.concat(mbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbes_sensors:
    all_sbes = []
else:
    all_sbes = pd.concat(sbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbp_sensors:
    all_sbp = []
else:
    all_sbp = pd.concat(sbp_sensors).drop_duplicates().reset_index(drop=True)

if not sss_sensors:
    all_sss = []
else:
    all_sss = pd.concat(sss_sensors).drop_duplicates().reset_index(drop=True)

In [20]:
# OPTIONAL:
# cut all instruments that are overachieving (measure more than twice the required maximum depth)
all_mbes = all_mbes.drop(all_mbes[(all_mbes.max_depth)> (survey_depth[1])*2].index)
sll_sbes = all_sbes.drop(all_sbes[(all_sbes.max_depth)> (survey_depth[1])*2].index)
all_sbp = all_sbp.drop(all_sbp[(all_sbp.max_depth)> (survey_depth[1])*2].index)

# Select Sensors with Space Restrictions

To receive recommended sensor models that respect the given space restrictions please run the following sections. 

In [21]:
for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        #print('Instruments for nautical survey: ')
        nautical_mbes = mbes_space(space_limits,nautical_mbes)
        nautical_sbes = sbes_space(space_limits, nautical_sbes)
        nautical_sbp = sbp_space(space_limits, nautical_sbp)
        
    if 'Seafloor Classification' in i:
        #print('Instruments for seabed investigation: ')
        seabed_mbes = mbes_space(space_limits, seabed_mbes)
        seabed_sbp = sbp_space(space_limits,seabed_sbp)
        seabed_sss = sss_space(space_limits, seabed_sss)
        
    if 'Oceanographic Survey' in i:
        #print('Instruments for investigation of the water column: '
        oceanic_mbes = mbes_space(space_limits,oceanic_mbes)
    
    if 'Geophysical Survey' in i:
        #print('Instruments for underground features: ')
        pass

In [22]:
# prepare lists for next step

mbes_sensors = []
sbes_sensors = []
sbp_sensors = []
sss_sensors = []

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        mbes_sensors.append(nautical_mbes)
        sbes_sensors.append(nautical_sbes)
        sbp_sensors.append(nautical_sbp)
        #sss_sensors.append(nautical_sss)
    if 'Seafloor Classification' in i:
        mbes_sensors.append(seabed_mbes)
        #sbes_sensors.append(seabed_sbes)
        sbp_sensors.append(seabed_sbp)
        sss_sensors.append(seabed_sss)
    if 'Oceanographic Survey' in i:
        mbes_sensors.append(oceanic_mbes)
        #sbes_sensors.append(oceanic_sbes)
        #sbp_sensors.append(oceanic_sbp)
        #sss_sensors.append(oceanic_sss)
    if 'Geophysical Survey' in i:
        #mbes_sensors.append(geophysical_mbes)
        #sbes_sensors.append(geophysical_sbes)
        #sbp_sensors.append(geophysical_sbp)
        #sss_sensors.append(geophysical_sss)
        pass

In [23]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates, only for not empty lists
if not mbes_sensors:
    all_mbes = []
else:
    all_mbes = pd.concat(mbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbes_sensors:
    all_sbes = []
else:
    all_sbes = pd.concat(sbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbp_sensors:
    all_sbp = []
else:
    all_sbp = pd.concat(sbp_sensors).drop_duplicates().reset_index(drop=True)

if not sss_sensors:
    all_sss = []
else:
    all_sss = pd.concat(sss_sensors).drop_duplicates().reset_index(drop=True)


In [24]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates
all_mbes = pd.concat([nautical_mbes, seabed_mbes, oceanic_mbes]).drop_duplicates().reset_index(drop=True)
all_sbes = pd.concat([nautical_sbes]).drop_duplicates().reset_index(drop=True)
all_sbp = pd.concat([nautical_sbp, seabed_sbp]).drop_duplicates().reset_index(drop=True)
all_sss = pd.concat([seabed_sss]).drop_duplicates().reset_index(drop=True)

## Results

Please be careful with comparing results. The results may vary depending on if all functions were called. 

### With the input restrictions, the following MBES models are recommended: 

In [25]:
all_mbes[['manufacturer','Model','min_depth','max_depth','freq_range','tx_length', 'tx_width','tx_height', 'tx_weight', 'rx_length', 'rx_width','rx_height','rx_weight']]

,manufacturer,Model,min_depth,max_depth,freq_range,tx_length,tx_width,tx_height,tx_weight,rx_length,rx_width,rx_height,rx_weight
0,Kongsberg,EM304 0.5x0.5,10.0,8000.0,26-34 kHz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Kongsberg,EM304 1x1,10.0,8000.0,26-34 kHz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Kongsberg,EM304 2x2,10.0,8000.0,26-34 kHz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Kongsberg,EM304 4x4,10.0,8000.0,26-34 kHz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Teledyne RESON,HydroSweepDS 1x2,10.0,11000.0,14-16 kHz,5658.0,299.0,155.0,0.0,299.0,2829.0,155.0,0.0
5,Teledyne RESON,HydroSweepDS 2x2,10.0,11000.0,14-16 kHz,2829.0,299.0,155.0,0.0,299.0,2829.0,155.0,0.0
6,Teledyne RESON,HydroSweepMD30 1x1,0.0,7000.0,24-30 kHz,2364.0,170.0,127.0,0.0,300.0,3195.0,127.0,0.0
7,Teledyne RESON,HydroSweepMD30 1x1.5,0.0,7000.0,24-30 kHz,2364.0,170.0,127.0,0.0,300.0,2170.0,127.0,0.0


### With the input restrictions, the following SBES models are recommended: 

In [26]:
all_sbes[['manufacturer','Model','min_depth','max_depth','freq_range','tx_length', 'tx_width','tx_height', 'tx_weight']]

,manufacturer,Model,min_depth,max_depth,freq_range,tx_length,tx_width,tx_height,tx_weight
0,Kongsberg,EA640,1.0,12500.0,12-500 kHz,213.0,438.0,84.0,5.0


### With the input restrictions, the following SBP models are recommended: 

In [27]:
all_sbp[['manufacturer','Model','min_depth','max_depth','freq_range','penetration_depth','freq_range_sec','tx_length', 'tx_width','tx_height', 'tx_weight','parametric']]

,manufacturer,Model,min_depth,max_depth,freq_range,penetration_depth,freq_range_sec,tx_length,tx_width,tx_height,tx_weight,parametric
0,Innomar,deep-36,5.0,6000,30-42 kHz,150.0,2-7 kHz,900.0,900.0,300.0,335.0,yes
1,Innomar,standard,0.5,500,85-115 kHz,50.0,4-15 kHz,260.0,520.0,340.0,30.0,yes
2,iXblue,Echoes3 3500 T7,1.0,6000,1.7-6 kHz,150.0,NaN,980.0,980.0,384.0,325.0,NaN
3,iXblue,Echoes 3 500 T3,1.0,1500,1.7-6 kHz,150.0,NaN,680.0,680.0,384.0,160.0,no
4,iXblue,Echoes 3 500 T1,1.0,200,1.7-6 kHz,150.0,NaN,450.0,450.0,384.0,57.0,no
5,Kongsberg,SBP29 3,10.0,11000,2-9 kHz,200.0,NaN,7450.0,800.0,350.0,1150.0,no
6,Kongsberg,SBP29 6,10.0,11000,2-9 kHz,200.0,NaN,3834.0,800.0,350.0,575.0,no
7,Kongsberg,SBP29 12,10.0,11000,2-9 kHz,200.0,NaN,1970.0,800.0,261.0,288.0,no
8,Kongsberg,TOPAS PS18,20.0,11000,15-21 kHz,200.0,0.5-6.0 kHz,1142.0,1140.0,260.0,525.0,yes
9,Kongsberg,TOPAS PS40,4.0,2000,35-45 kHz,75.0,1-10 kHz,840.0,534.0,118.0,80.0,yes


### With the input restrictions, the following SSS models are recommended: 

In [28]:
all_sss[['manufacturer','Model','min_depth','depth_rating','max_depth','freq_range','tx_length','tx_weight']]

,manufacturer,Model,min_depth,depth_rating,max_depth,freq_range,tx_length,tx_weight
